### 1、电影海报保存

In [1]:
# -*- coding: utf-8 -*-

import requests
from PIL import Image
from io import BytesIO
import os

# 保存图片
def saveimgs(img_cont, img_name):
    imgs_path = "E:/jupyter/TK02/pics/"
    if not os.path.exists(imgs_path):
        os.makedirs(imgs_path)
        print('目录创建成功！')
    try:
        img = Image.open(BytesIO(img_cont))
        img.save('%s%s.jpg' % (imgs_path, img_name))
        return('保存成功！')
    except Exception as err:
        return('保存失败,错误内容：' + str(err))

In [2]:

mv_url = 'https://api.douban.com/v2/movie/1292052?apikey=0df993c66c0c636e29ecbb5344252a4a'
mv_req = requests.get(mv_url, verify=False)
mv_req.encoding = 'utf8'
mv_rep = mv_req.json()
print(mv_rep)
if mv_req.status_code == 200 and mv_rep['image']:
    img_url = mv_rep['image']
    img_name = mv_rep['title']
    img_req = requests.get(img_url, verify=False)
    if img_req.status_code in (200, 304):
        img_cont = img_req.content
        saveimg = saveimgs(img_cont, img_name)
        print(saveimg)
    else:
        print('找不到图片')
else:
    print('请求出错...')

D:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
D:\Anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'rating': {'max': 10, 'average': '9.7', 'numRaters': 1571440, 'min': 0}, 'author': [{'name': '弗兰克·德拉邦特 Frank Darabont'}], 'alt_title': '肖申克的救赎 / 月黑高飞(港)', 'image': 'https://img3.doubanio.com/view/photo/s_ratio_poster/public/p480747492.jpg', 'title': 'The Shawshank Redemption', 'summary': '20世纪40年代末，小有成就的青年银行家安迪（蒂姆·罗宾斯 Tim Robbins 饰）因涉嫌杀害妻子及她的情人而锒铛入狱。在这座名为肖申克的监狱内，希望似乎虚无缥缈，终身监禁的惩罚无疑注定了安迪接下来灰暗绝望的人生。未过多久，安迪尝试接近囚犯中颇有声望的瑞德（摩根·弗里曼 Morgan Freeman 饰），请求对方帮自己搞来小锤子。以此为契机，二人逐渐熟稔，安迪也仿佛在鱼龙混杂、罪恶横生、黑白混淆的牢狱中找到属于自己的求生之道。他利用自身的专业知识，帮助监狱管理层逃税、洗黑钱，同时凭借与瑞德的交往在犯人中间也渐渐受到礼遇。表面看来，他已如瑞德那样对那堵高墙从憎恨转变为处之泰然，但是对自由的渴望仍促使他朝着心中的希望和目标前进。而关于其罪行的真相，似乎更使这一切朝前推进了一步……\n本片根据著名作家斯蒂芬·金（Stephen Edwin King）的原著改编。', 'attrs': {'pubdate': ['1994-09-10(多伦多电影节)', '1994-10-14(美国)'], 'language': ['英语'], 'title': ['The Shawshank Redemption'], 'country': ['美国'], 'writer': ['弗兰克·德拉邦特 Frank Darabont', '斯蒂芬·金 Stephen King'], 'director': ['弗兰克·德拉邦特 Frank Darabont'], 'cast': ['蒂姆·罗宾斯 Tim Robbins', '摩根·弗里曼 Morgan Freeman', '鲍勃·冈顿 Bob Gunton', '

### 2、豆瓣电影 top 250电影信息保存 

In [1]:
import requests
import os
from PIL import Image
from io import BytesIO
import csv
import time
import datetime
import random

def saveimgs(img_cont, img_name):
    imgs_path = "E:/jupyter/TK02/pics/"
    if not os.path.exists(imgs_path):
        os.makedirs(imgs_path)
        print('目录创建成功！')
    try:
        img = Image.open(BytesIO(img_cont))
        img.save('%s%s.jpg' % (imgs_path, img_name))
        return('保存成功！')
    except Exception as err:
        return('保存失败,错误内容：' + str(err) + '错误所在的行号：'+ str(err.__traceback__.tb_lineno))

# def count_time(func):
#     def int_time(*args, **kwargs):
#         start_time = datetime.datetime.now()  
#         func()
#         over_time = datetime.datetime.now()   
#         total_time = (over_time-start_time).total_seconds()
#         print('程序共计%s秒' % total_time)
#     return int_time

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Connection': 'keep-alive',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
}

date = time.strftime("%Y%m%d", time.localtime())
csv_header = ['排名','电影名称', '豆瓣评分', '导演', '主演', '电影类型', '上映时间', '片长', 'alt', '图片地址']
try:
    print('开始运行')
    with open('%s.csv' % date, 'w+', newline='') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(csv_header)
        count = 0
        for i in range(0, 241, 20):
            url = 'https://api.douban.com/v2/movie/top250?start=%d&apikey=0df993c66c0c636e29ecbb5344252a4a' % i
            req = requests.get(url, headers = headers)
            req.encoding = 'utf8'
            rep = req.json()['subjects']
    #         print(rep)
            if req.status_code == 200 and rep:
                for j in rep:
                    count += 1
                    directors = ','.join([str(dt['name']) for dt in j['directors']])
                    casts = ','.join([str(cast['name']) for cast in j['casts']])
                    img_url = (''.join(j['images']['medium'])).replace('.webp', '.jpg')
                    row = [
                        [
                            count,
                            j['title'],
                            '%.2f' % j['rating']['average'],
                            directors,
                            casts,
            #                 j['genres'],
                            ','.join(str(g) for g in j['genres']),
            #                 j['pubdates'],
                            ','.join(str(pd) for pd in j['pubdates']),
            #                 j['durations'],
                            ','.join(str(dt) for dt in j['durations']),
                            j['alt'],
                            img_url,
                        ]
                    ]
                    f_csv.writerows(row)  
                    # 保存图片
                    img_req = requests.get(img_url) 
                    img_req.encoding = 'utf8'
                    if img_req.status_code in (200, 304):
                        img_cont = img_req.content
                        saveimg = saveimgs(img_cont, j['title'])
                        print(saveimg)
                    else:
                        print('图片访问失败')
                time.sleep(random.ranint(1,3))
            else:
                print(req.content)
        print('finished!')
except Exception as err:
    print('错误所在的行号：', err.__traceback__.tb_lineno)
    print('打开错误，' + str(err))
    

开始运行
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！
保存成功！